## Importing Neccessary Libraries

In [1]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, tool, AgentExecutor

from dotenv import load_dotenv
import re
from importlib import import_module
import subprocess
import sys
import tempfile
import os

In [2]:
load_dotenv()

True

## Load Model

In [67]:
# Template for the system and user messages
template = ChatPromptTemplate.from_messages([
    ("system", """You are an AI Python code generator and tester. Your task is to generate and execute Python code following a structured process. Follow these steps carefully:

    1. **Code Generation:**
       - Generate Python code to complete the task based on the user's input. Include test cases for validation where appropriate.

    2. **Library Installation:**
       - If the code requires any external libraries, use the install_libraries function to install them first. For example, use `install_libraries("python-pptx")` to install the necessary library.

    3. **Code Execution:**
       - Once the code is generated and necessary libraries are installed, use the `execute_code` function to run the code.
       - If the code runs successfully without errors and the output includes "Execution successful", respond with the final code and stop further execution.

    4. **Error Handling:**
       - If an error occurs during code execution (e.g., syntax errors, exceptions), debug the issue and provide both the error message and the fix.
       - Reattempt execution after fixing the error.
       
    **Important Notes:**
    - Do not repeat successful tasks. Once the task completes successfully, stop execution.
    - Provide clear error messages and details if debugging is necessary.

    Available tools:
    - install_libraries(libraries: str) -> str
    - execute_code(code: str) -> str
    """),
    ("user", """Generate Python code to {input} {agent_scratchpad}"""),
])

# Load the model
llm = ChatGroq(model="gemma2-9b-it", temperature=0.5)

## Create Tools

In [68]:
@tool
def install_libraries(libraries: str) -> str:
    """Install necessary libraries. Input should be a comma-separated string of library names."""
    libraries = [lib.strip() for lib in libraries.split(',')]
    results = []
    
    for lib in libraries:
        # Try importing the module in different ways
        try:
            # Attempt to import the library normally
            import_module(lib.replace('-', '_'))
            results.append(f"{lib} is already installed.")
        except ImportError:
            # Try importing after removing "python-" prefix if present
            try:
                import_module(lib.replace('python-', ''))
                results.append(f"{lib} (after modifying the name) is already installed.")
            except ImportError:
                # If import fails, try installing the library via pip
                try:
                    subprocess.check_call([sys.executable, "-m", "pip", "install", lib], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                    results.append(f"{lib} was successfully installed.")
                except subprocess.CalledProcessError as e:
                    results.append(f"Failed to install {lib}: {e}")
    
    return "\n".join(results)

@tool
def execute_code(code: str) -> str:
    """Execute the code and return output and error as a single string."""
    with tempfile.NamedTemporaryFile(mode='w', suffix=".py", delete=False) as temp_file:
        temp_file.write(code)
        temp_file_path = temp_file.name
    
    try:
        # Run the code and capture output and errors
        result = subprocess.run([sys.executable, temp_file_path], capture_output=True, text=True, timeout=60)
        output = result.stdout
        err = result.stderr
        
        if err:
            return f"Execution failed with error:\n{err}"
        elif output:
            return f"Execution successful. Output:\n{output}"
        else:
            return "Execution completed without output."
    
    except subprocess.TimeoutExpired:
        return "Execution timed out after 60 seconds."
    
    except Exception as e:
        return f"An error occurred during execution: {str(e)}"
    
    finally:
        # Ensure the temporary file is deleted after execution
        os.unlink(temp_file_path)

In [69]:
# Create tools list
tools = [install_libraries, execute_code]

# Create agent
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=template
)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5)

In [70]:
try:
    # Invoke the agent with the task input
    result = agent_executor.invoke({
        "input": "make a detailed deck on the best forms of leadership with atleast 5 slides having 100 words in every slide and save it to a pptx called leadership.pptx"
    })
    print(result)
except Exception as e:
    print(f"An error occurred during agent execution: {str(e)}")



> Entering new AgentExecutor chain...

Invoking: `install_libraries` with `{'libraries': 'python-pptx'}`


python-pptx (after modifying the name) is already installed.
Invoking: `execute_code` with `{'code': "import pptx\n\nprs = pptx.Presentation()\n\nslide_1 = prs.slides.add_slide(prs.slide_layouts[0])\ntitle = slide_1.shapes.title\ntitle.text = 'Best Forms of Leadership'\n\nslide_2 = prs.slides.add_slide(prs.slide_layouts[1])\ntitle = slide_2.shapes.title\ntitle.text = 'Transformational Leadership'\nslide_2.shapes.placeholders[1].text = 'Inspire and motivate followers to achieve great things. Focus on vision, empowerment, and ethical behavior.'\n\nslide_3 = prs.slides.add_slide(prs.slide_layouts[1])\ntitle = slide_3.shapes.title\ntitle.text = 'Servant Leadership'\nslide_3.shapes.placeholders[1].text = 'Prioritize the needs of others, empower them, and build strong relationships. Focus on service, empathy, and stewardship.'\n\nslide_4 = prs.slides.add_slide(prs.slide_layouts[1])\nt

In [71]:
print(result["output"].split("```")[0])


Agent stopped due to max iterations.
